In [1]:
import pandas as pd

In [ ]:
results = pd.read_csv("results.csv", index_col=0, sep="\t")
finetuned_results = pd.read_csv("finetuned_results.csv", index_col=0, sep="\t")

,latin_silver_target_bowphs_GreBerta,latin_silver_target_bowphs_LaBerta,latin_silver_target_bowphs_PhilBERTa,latin_silver_target_bowphs_SPhilBERTa,latin_silver_target+random_bowphs_GreBerta,latin_silver_target+random_bowphs_LaBerta,latin_silver_target+random_bowphs_PhilBERTa,latin_silver_target+random_bowphs_SPhilBERTa,latin_gold_target_bowphs_GreBerta,latin_gold_target_bowphs_LaBerta,...,greek+latin_silver_target+random_bowphs_PhilBERTa,greek+latin_silver_target+random_bowphs_SPhilBERTa,greek+latin_gold_target_bowphs_GreBerta,greek+latin_gold_target_bowphs_LaBerta,greek+latin_gold_target_bowphs_PhilBERTa,greek+latin_gold_target_bowphs_SPhilBERTa,greek+latin_gold_target+random_bowphs_GreBerta,greek+latin_gold_target+random_bowphs_LaBerta,greek+latin_gold_target+random_bowphs_PhilBERTa,greek+latin_gold_target+random_bowphs_SPhilBERTa
num_ret,50000.000000,50000.000000,50000.000000,50000.000000,100500.000000,100500.000000,100500.000000,100500.000000,2000.000000,2000.000000,...,100500.000000,100500.000000,2000.000000,2000.000000,2000.000000,2000.000000,20100.000000,20100.000000,20100.000000,20100.000000
num_rel,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,100.000000,100.000000,...,500.000000,500.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
num_rel_ret,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,100.000000,100.000000,...,500.000000,500.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
num_q,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,20.000000,20.000000,...,100.000000,100.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
map,0.687870,0.893462,0.365067,0.954387,0.824218,0.965325,0.584335,0.956545,0.377993,0.637578,...,0.227542,0.935936,0.154442,0.265502,0.114272,0.730852,0.010138,0.023282,0.011145,0.605347
gm_map,0.623790,0.862516,0.205952,0.947895,0.774044,0.960798,0.483770,0.950732,0.302895,0.546689,...,0.096356,0.921691,0.106963,0.170627,0.095232,0.687407,0.008755,0.015476,0.009285,0.494872
bpref,0.640000,0.869200,0.336800,0.937600,0.788400,0.956400,0.516000,0.943200,0.304000,0.582000,...,0.197200,0.918000,0.068000,0.192000,0.058000,0.654000,0.000000,0.000000,0.000000,0.540000
Rprec,0.632000,0.864000,0.344000,0.922000,0.764000,0.932000,0.514000,0.930000,0.330000,0.580000,...,0.218000,0.896000,0.090000,0.230000,0.080000,0.640000,0.000000,0.000000,0.000000,0.560000
recip_rank,0.967833,0.973333,0.751059,0.995000,0.972833,1.000000,0.869706,0.995000,0.661865,0.904545,...,0.431217,0.993333,0.211281,0.376643,0.249837,0.900000,0.006543,0.021624,0.023915,0.815278
P@5,0.632000,0.864000,0.344000,0.922000,0.764000,0.932000,0.514000,0.930000,0.330000,0.580000,...,0.218000,0.896000,0.090000,0.230000,0.080000,0.640000,0.000000,0.000000,0.000000,0.560000


In [ ]:
# make table good looking in latex
t_results = results.T.copy()
for idx, i in enumerate(["lang", "split", "setting", "author", "model"]):
    newcol = []
    for j in t_results.index.str.split("_"):
        newcol.append(j[idx])
    t_results[i] = newcol
t_results = t_results.reset_index(drop=True).set_index(["lang", "split", "setting", "model"])
t_results = t_results.drop("author", axis=1)
t_results.columns = ["\\rotatebox{{90}}{{{col_name}}}".format(col_name=col.replace("_", "-")) for col in t_results.columns]
t_results = t_results.loc[:, ~t_results.columns.str.contains("num")]
col_format = "l" * 4 + "c" * (len(t_results.columns))
col_format = col_format[:-18] + "|" + col_format[-18:-9] + "|" + col_format[-9:]  # Add vertical line before the last 8 columns
formatters = {
        **{
            i: lambda x: f"{x:.2f}" for i in t_results.columns if "P@" not in i and "NDCG" not in i
            },
        **{
            i: lambda x: "\gradient{" + f"{x:.2f}" + "}"
            for i in t_results.columns if "P@" in i or "map" in i or "recip-rank" in i
            or "bpref" in i or "Rprec" in i
            },
        **{
            i: lambda x: r"\newgradient{" + f"{x:.2f}" + "}"
            for i in t_results.columns if "NDCG" in i
            },
    }

def save_to_latex(results, file_name, col_format, formatters):
    latex_str = results.to_latex(
        index=True,
        float_format="%.3f",
        column_format=col_format,
        formatters=formatters,
    )
    latex_str = (latex_str.replace(r'\toprule', '').replace("cline", "cmidrule"))

    os.makedirs("tables", exist_ok=True)
    with open(f"tables/{file_name}", "w") as f:
        f.write(latex_str)

save_to_latex(t_results, "results.tex", col_format, formatters)

In [ ]:
col_to_keep = ["{map", "recip-rank", "P@5}", "P@10}", "NDCG@5}", "NDCG@10}"]
col_to_keep = [i for i in t_results.columns if any(j in i for j in col_to_keep)]
lang_col_format = "l" * 3 + "c" * (len(col_to_keep))
latin_pretrained_results = t_results.loc[("latin"), col_to_keep]
save_to_latex(latin_pretrained_results, "latin_pretrained_results.tex", lang_col_format, formatters)
greek_pretrained_results = t_results.loc["greek", col_to_keep]
save_to_latex(greek_pretrained_results, "greek_pretrained_results.tex", lang_col_format, formatters)
latin_greek_pretrained_results = t_results.loc[("latin+greek"), col_to_keep]
save_to_latex(latin_greek_pretrained_results, "latin_greek_pretrained_results.tex", lang_col_format, formatters)
greek_latin_pretrained_results = t_results.loc[("greek+latin"), col_to_keep]
save_to_latex(greek_latin_pretrained_results, "greek_latin_pretrained_results.tex", lang_col_format, formatters)

In [ ]:
t_finetuned_results = finetuned_results.T.copy()
for idx, i in enumerate(["lang", "split", "setting", "author", "model"]):
    newcol = []
    for j in t_finetuned_results.index.str.split("_"):
        newcol.append(j[idx])
    t_finetuned_results[i] = newcol
t_finetuned_results = t_finetuned_results.reset_index(drop=True).set_index(["lang", "split", "setting", "model"])
t_finetuned_results = t_finetuned_results.drop("author", axis=1)
t_finetuned_results.columns = ["\\rotatebox{{90}}{{{col_name}}}".format(col_name=col.replace("_", "-")) for col in t_finetuned_results.columns]

t_finetuned_results = t_finetuned_results.loc[:, ~t_finetuned_results.columns.str.contains("num")]
col_format = "l" * 4 + "c" * (len(t_finetuned_results.columns))
col_format = col_format[:-18] + "|" + col_format[-18:-9] + "|" + col_format[-9:]  # Add vertical line before the last 8 columns
save_to_latex(t_finetuned_results, "finetuned_results.tex", col_format=col_format, formatters=formatters)
# t_finetuned_results

In [ ]:
save_to_latex(
    t_finetuned_results.loc[:, col_to_keep],
    "selected_finetuned_results.tex",
    col_format="l" * 4 + "c" * len(col_to_keep),
    formatters=formatters
)

In [ ]:
latin_finetuned_results = t_finetuned_results.loc[("latin"), col_to_keep]
save_to_latex(latin_finetuned_results, "latin_finetuned_results.tex", lang_col_format, formatters)
greek_finetuned_results = t_finetuned_results.loc["greek", col_to_keep]
save_to_latex(greek_finetuned_results, "greek_finetuned_results.tex", lang_col_format, formatters)
latin_greek_finetuned_results = t_finetuned_results.loc[("latin+greek"), col_to_keep]
save_to_latex(latin_greek_finetuned_results, "latin_greek_finetuned_results.tex", lang_col_format, formatters)
greek_latin_finetuned_results = t_finetuned_results.loc[("greek+latin"), col_to_keep]
save_to_latex(greek_latin_finetuned_results, "greek_latin_finetuned_results.tex", lang_col_format, formatters)